### Imports utilizados

In [55]:
import requests
import time
import random
import re
import pandas as pd
import os
from tqdm import tqdm as tqdm
from bs4 import BeautifulSoup

### Parsing da Página Principal da Wikipedia

Cria o diretório para armazenar os verbetes, percorre todo o conjunto e salva o HTML

In [56]:


def get_infobox(url):
    e = requests.get(url)
    soup = BeautifulSoup(e.text, 'html.parser') 
    tables = soup.findAll('table')
    infobox = ""
    file_title = url.split("/")[-1]
    if not os.path.exists(f'verbetes/{file_title}'):
        os.makedirs(f'verbetes/{file_title}')
    for table in tables:
        if not table.has_attr("class"):
            print(f"Infobox não encontrada para (sem classe) {url}")
            return None
        if "infobox" in table["class"] or "infobox_v2" in table["class"]:
            infobox = table
            break
    if infobox == "":
        print(f"Infobox não encontrada para {url}")
        return None
    chava = {}
#dando erro. Provavelmente por causa da forma como o título está sendo pego - lavie
    titulo = infobox.findAll("th")
    if len(titulo) == 0:
        print(f"Infobox não encontrada para (titulo vazio) {url}")
        return None
    #print(titulo)
    titulo = titulo[0]
    for tr in soup.findAll('tr'):
        if len(tr.findAll("td")) == 2:
            messi = tr.findAll("td")
            chava[messi[0].text.strip()] = [li.text.strip() for li in messi[1].findAll("li")] if messi[1].findAll("li") else messi[1].text.strip()
            
    #print(chava)
    chava["Título"] = titulo.text.strip()
    chava_df = pd.DataFrame.from_dict(chava, orient='index')
    chava_df.transpose().to_json(f'verbetes/{file_title}/{file_title}.json'.replace(" ","_").replace(":",""), orient='index')
    #print(chava_df.transpose())

        

In [57]:
def cria_pasta(verbet):
    os.makedirs('verbetes', exist_ok=True)
    titulo = verbet.split('/')[-1]
    url = f"https://pt.wikipedia.org{verbet}"
    a = requests.get(url)
    if a.status_code != 200:
        print(f"Erro ao acessar {url}")
        return
    if a.status_code == 200:
        soup = BeautifulSoup(a.text, 'html.parser')
        os.makedirs(f'verbetes/{titulo}', exist_ok=True)
        with open(f'verbetes/{titulo}/{titulo}.html', 'w') as f:
            f.write(soup.prettify())
        time.sleep(random.randint(0,1))

In [58]:
a = requests.get('https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal')
soup = BeautifulSoup(a.text, 'html.parser')

Para cada link na seção "main" da página principal, verifica se o link representa um verbete, caso positivo, adiciona à um Set para evitar duplicatas

In [59]:
links = soup.main.find_all('a')
all_verbets = set()
for link in links:
    if link.has_attr('href') and re.match(r'/wiki/.*', link['href']) and not re.match(r'/wiki/.*:.*', link['href']):
            all_verbets.add(link['href'])

Escolhe aleatoriamente um verbet da conjunto obtido da página principal, e faz o mesmo processamento. O loop é executado até que o conjunto de verbetes tenha 5000 verbetes.

In [ ]:
while(len(all_verbets) < 5000):
    verbet = random.choice(list(all_verbets))
    a = requests.get('https://pt.wikipedia.org' + verbet)
    soup = BeautifulSoup(a.text, 'html.parser')
    links = soup.find_all('a')
    for link in tqdm(links):
        if link.has_attr('href') and re.match(r'/wiki/.*', link['href']) and not re.match(r'/wiki/.*:.*', link['href']):
            all_verbets.add(link['href'])
            cria_pasta(link['href'])
            get_infobox('https://pt.wikipedia.org' + link['href'])
            
    time.sleep(random.randint(1,2))

 21%|██        | 108/514 [00:03<00:15, 25.74it/s]

Infobox não encontrada para (titulo vazio) https://pt.wikipedia.org/wiki/Al-Shabab_Riyadh


 23%|██▎       | 120/514 [00:04<00:20, 18.99it/s]

Infobox não encontrada para https://pt.wikipedia.org/wiki/Estandarte_Negro
Infobox não encontrada para (titulo vazio) https://pt.wikipedia.org/wiki/2002


 24%|██▍       | 125/514 [00:11<01:24,  4.61it/s]

...

Infobox não encontrada para https://pt.wikipedia.org/wiki/Terrorismo


 26%|██▌       | 132/514 [00:14<02:04,  3.07it/s]

Infobox não encontrada para https://pt.wikipedia.org/wiki/Fundamentalismo_isl%C3%A2mico


 27%|██▋       | 137/514 [00:16<00:46,  8.16it/s]


KeyboardInterrupt: 